In [15]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from helpers.rotation import RotateFromTo

from yaml import load
from yaml.loader import Loader


In [16]:
BASENAME = "11_Prerouge"

fp = f"../../real_data/{BASENAME}/params.yaml"

params = load(open(fp), Loader)

GLOBAL_SHIFT = params["alignment"]["globalShift"]

PASSAGE = "Chapelle"
SECTION = 1

In [17]:
data = pv.read(f"../../real_data/{BASENAME}/output/{PASSAGE}_{SECTION}_clean.stl").extract_surface().triangulate()

In [18]:
plane_origins = np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_{SECTION}_Planes_Origin.txt") + GLOBAL_SHIFT
plane_vectors =  np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_{SECTION}_Planes.txt")

In [19]:
# define orthogonal unit vectors in horizontal plane 
d0 = np.array([1,0,0])
d1 = np.array([0,1,0])

# project the section-normal vectors onto horizontal plane. 
horiz_dip_dir = np.asarray((plane_vectors.dot(d0), plane_vectors.dot(d1), np.zeros(len(plane_vectors)))).T

In [20]:
print(len(horiz_dip_dir))

129


In [21]:
for n, (orig, vec, vec_horiz) in tqdm(enumerate(zip(plane_origins, plane_vectors, horiz_dip_dir)), total = len(plane_origins)):
    plane = pv.Plane(orig, vec, 20, 20, 100, 100).extract_surface().triangulate()
    inter, _, _ = data.intersection(plane, split_first = False, split_second = False)

    R1 =  RotateFromTo(vec, vec_horiz, verbose=False)
    pp = inter.points - np.mean(plane.points, axis=0)
    pp_vertical = R1 @ pp.T
    R2 = RotateFromTo(vec_horiz, d1)
    pp_rotated = R2 @ pp_vertical

    np.savetxt(f"../../real_data/{BASENAME}/process/sections_as_polylines/{PASSAGE}_{SECTION}_unordered_section_{n:03d}.txt", pp_rotated.T)
        
    #pp_reordered = order_points(pp_rotated.T.tolist(), 0)


  3%|██▌                                                                               | 4/128 [00:15<07:56,  3.85s/it]


KeyboardInterrupt: 